In [36]:
import pandas as pd
import naya
import json
from itertools import islice

from google.cloud import bigquery

In [40]:
#conversion du fichier json en json line delimiter pour utiliser pd.read_json avec une chunk size. (Nombre de ligne processée en une fois)
#!cat ../raw_data/export_23-5_minified.json | jq -c '.[]' > ../raw_data/export_converted_2.json

In [26]:
#!pip install bigjson

In [28]:
#Conversion du Json en JsonL 3h40 d'execution
#import bigjson
# Reads json file in streaming mode
#with open('../raw_data/export_23-5_minified.json', 'rb') as f:
#    json_data = bigjson.load(f)
#
#    # Open output file  
#    with open('../raw_data/export_converted.jsonl', 'w') as outfile:
#        # Iterates over input json
#        for data in json_data:
#            # Converts json to a Python dict  
#            dict_data = data.to_python()
#            
#            # Saves the output to output file
#            outfile.write(json.dumps(dict_data)+"\n")

In [35]:
#Nombre de lignes totales dans le JSON : 1320229

94302.07142857143

In [61]:
#fp = open('../raw_data/export_23-5_minified.json', 'r')
file_path = '../raw_data/export_converted_2.json'
df = pd.read_json(file_path, lines=True, chunksize=94303) #Chunk size pour avoir 14 chunk

In [ ]:
keys_1 = ['event_id','user_properties']
keys_2 = ['Newsletter','Products','Pieces','Language','Type','Country','Games','Purchase']

In [ ]:
#Localisation du projet name et table_id sur gcp
project="marbotic"
table_id = "marbotic_dataset.event_properties"

In [ ]:
#intégration des credentials 
from google.oauth2 import service_account

key_path = "/Users/antonin/code/AntoninAnq/gcp/marbotic-7d02fac30bd8.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [71]:
for c in df:
    event_properties = c[['event_id', 'event_properties']].copy()
    print(event_properties['event_properties'].keys())
    break

RangeIndex(start=848727, stop=943030, step=1)


In [ ]:
fp = open('../raw_data/export_23-5_minified.json', 'r')

messages = naya.stream_array(naya.tokenize(fp))

In [ ]:
keys_1 = ['event_id','event_properties']
keys_2 = ['[Game] Code','[Profile] Age (days)','[Scene] Name','[Game] Language','[Scaffolding] Level','[Time] Spent','[Activity] Name','[Activity] Piece Stamped','[Activity] Piece Asked','[Activity] Modality',
         '[Activity] Nb Wrong Answer','[Game] Piece Code','[Scene] Section']

keys_2[3]

In [ ]:
#prepare json file : a list of dicts
def create_json_file(sample,step):
    json_1 = []
    for i in range(len(sample)):
        dico = {}
        for key in keys_1:
            if key == 'event_id':
                dico[key] = sample[i][key]
            else :
                for k in keys_2:
                    try :
                        dico[k] = sample[i][key][k]
                    except:
                        dico[k] = ""
        json_1.append(dico)
    with open(f'../marbotic/data/user_properties_{step}.json', 'w') as f:
        json.dump(json_1, f, indent=None)  

In [ ]:
#extract 100 lines
step = 100000

for i in range(11,14):
    if i < 13:
        sample = []
        for m in islice(messages,i*step,(i*step)+step):
            sample.append(m)
        create_json_file(sample,i)
    else:
        for m in islice(messages,i*step,None):
            sample.append(m)
        create_json_file(sample,i)